Tests et Profilage, Python
==========================

Le présent tutoriel vise à éclaircir certains concepts reliés aux tests en
programmation et à mettre l'emphase sur les bonnes pratiques d'assurance
qualité en Python. Nous allons également investiguer comment profiler son code
afin de déterminer la performance.

Tests unitaires
---------------
Le pilier principal de l'assurance qualité en programmation
est les tests unitaires. Les tests unitaires ont comme objectif de vérifier que
mon code fait ce qu'il est censé faire.

Les tests unitaires sont composés de 4 étapes:
1. Préparation (`setUp`): les composantes nécessaires pour le test sont
   instanciées.
2. Exercice: L'unité de code testée est lancée.
3. Vérification: Le sortant de l'unité de code en question est comparé à sa
   valeur anticipée.
4. Ramassage (`tearDown`): les composantes nécessaires pour le test sont
   détruites.

Un test unitaire est généralement conçu afin d'isoler une unité de code. Il
est important de découpler les tests unitaires.   Nous allons commencer par
nous habituer à la structure générale des tests. Le premier exemple est le
script `utilities.py` et sa classe de tests dans `test_utilities.py`.  
Nous n'allons pas aborder le script `utilities.py` dans ce tutoriel puisqu'il nécessite des concepts de base en programmation orienté objet. Cependant, je le laisse à titre d'exemple puisque vous allez probablement fréquemment rencontrer des exemples similaires. 

Pytest
------

Pytest nous permet de lancer nos tests et de voir le résultat
avec un résumé graphique sur la ligne de commande. La commande est `pytest
<votre_fichier_de_test.py> `.

Il y a plusieurs options utiles pour commencer
tels que `-s` qui permet d'afficher le _standard out_ (les print statements,
sinon ils ne sont pas imprimés puisque pytest capture l'ensemble des sortants
des tests.

Pytest est un des principaux modules de test dans Python. Il supporte les tests
unitaires classiques comme nous venons juste de voir. De plus, il permet de
gérer les tests différemment avec des _fixture_. Nous allons refactoriser la
classe TestUtilities afin d'utiliser les _fixture_.
Le ficher est *test_utilities_refactored.py*

In [ ]:
@pytest.fixture(scope='module')
def prepare():
    print('nothing to prepare in this example!')
    return 

def test_translate_to_jadensmith_raises_value_error(prepare):
    with pytest.raises(ValueError):
        ut.translate_to_jadensmith(999)

def test_translate_to_jadensmith_single_word(prepare):
    sentence = "bonjour"
    expected_value = "Bonjour"

    actual_value = ut.translate_to_jadensmith(sentence)

    assert expected_value == actual_value


Les _fixtures_ sont une fonctionnalité très puissante de pytest.

Pytest Coverage
---------------

_pytest-cov_ est un module supplémentaire nous permettant de visualiser la
proportion du code testée. Il est facile de se restreindre à seulement quelques
chemins possibles par le code. Par exemple. si j'ai le code suivant:

In [ ]:
if first_name == "":
   if last_name == "":
        db.set_name("unknown")
   else:
        db.set_name(last_name)
else:
    if last_name == "":
        db.set_name(first_name + "_unknown")
    else:
        db.set_name(first_name +  " " + last_name)

Nous devons écrire soit 4 tests ou bien un test parcourant les 4 chemins.
Un module comme _pytest-cov_ nous permet de calculer la proportion du code qui
est réellement testé.

La commande est `pytest --cov=<nom du module> <test_directory>`

![pytest cov result](./include/pytest_cov.png)

La photo précédente démontre le résultat d'une couverture de test.

Analyse comparative en Python
-----------------------------
Il est parfois important de respecter certains critères de performance.
Comment valider les performances de composantes précises de notre software?
Pour des fonctions isolées, nous pouvous utiliser le module `timeit`.

In [ ]:
import timeit
s = """
with open("sherlock.txt") as f:
    lines = f.readlines()
meme_lines = [ut.translate_to_internet_meme(l) for l in lines[:100]]
"""
print(timeit.timeit(stmt=s, setup="from __main__ import ut", number=1000))

Le code précédent ouvre un ficher texte (Sherlock Holmes) et appelle la fonction `translate_to_internet_meme` 
sur les 100 premières phrases. Cette procédure est repétée 1000 fois. Cela permet d'obtenir 
une moyenne assez précise des performances.   
Noter que le code est sous forme de `str` ce qui limite quelque peut les possibilités d'essaies.

Profilage en Python
-------------------
Outre l'analyse comparative, le profilage peut également s'avérer utile afin de mieux comprendre son programme.
Un profilage est un ensemble de statistiques descriptives du temps d'éxécution des divers composantes d'un programme.

Le module le plus populaire et utile est sans doute le module `cprofile`.
Ce module est fait pour créer une profile d'exécution et non afin d'analyse comparative, utiliser `timeit`dans ce contexte.

La commande pour lancer le profileur *cprofile* est la suivante:
`python -m cprofile <your_script_name.py`

In [ ]:
import time

def f(x):
    x = str(x)
    c = "this value is " + x +  "."
    time.sleep(1)
    print(c)

l = [1,2,3,4,5,6,7]

for v in l:
    f(v)

![cprofile](./include/result_cprofile.png)

Voici la signification des colonnes:
* ncalls: le nombre de fois la fonction a été appelé.
* tottime: le temps total passé dans une fonction.
* percall: tottime / ncalls
* cumtime: temps cumulatif passé dans une fonction et ses sous-fonctions.
